### Let's make some small multiple maps!

Let's see how things have changed overtime with small multiples.

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import mapclassify
import numpy as np
import os

First let's read in our data and our shapefile.

In [ ]:
df = pd.read_csv('data/opiate_deaths.csv', dtype={'INJURY_FIPS':str})

mn_cnty = gpd.read_file('gis/tl_2015_mn_county/tl_2015_mn_county.shp')

In [ ]:
df['DEATHDATE'] = pd.to_datetime(df['DEATHDATE'])
df['death_year'] = df['DEATHDATE'].apply(lambda x: x.year)

We need a list of all of the years we'll be using here.

In [ ]:
year_list = df['death_year'].unique().tolist()
year_list

Now we need to create a pivot table so that we have a death count for each year and each county.

In [ ]:
by_yearcnty = pd.pivot_table(df, values='STATEID', index='INJURY_FIPS',
                            columns=['death_year'], aggfunc='count').reset_index()
by_yearcnty

Next, we're going to merge our year-by-year county data with our shapefile. We join the _data to the shapefile_ instead of joining the _shapefile to the data_ so that all the counties will be represented on our maps, even if they don't have death data for a given year. Otherwise, we would have a swiss cheese map.

In [ ]:
yr_geodeaths = mn_cnty.merge(by_yearcnty, left_on='GEOID', right_on='INJURY_FIPS', how='left')

yr_geodeaths = yr_geodeaths.fillna({2005:0,2006:0,2007:0,2008:0,2009:0,2010:0,
                                   2011:0,2012:0,2013:0,2014:0,2015:0,
                                   2016:0,2017:0})

yr_geodeaths

Now we're going to write a loop that will take each year in our `year_list` and grab the associated column in our pivot table data and map a map using that column.

In [ ]:
plt.figure(figsize=(20,15), facecolor='white')

plot_number = 1
for year in year_list:
    year = year
    # Inside of an image that's a 15x13 grid, put this
    # graph in the in the plot_number slot.
    ax = plt.subplot(4, 4, plot_number)
    vmin = 0#yr_geodeaths[year].min
    vmax = 10#yr_geodeaths[year].max
    
    yr_geodeaths.plot(column=year, cmap='Greens', linewidth=0.8, vmin=vmin, 
                      vmax=vmax, edgecolor='0.8',ax=ax, legend=True)
    
    ax.axis('off')
    
    ax.set_title(year, fontdict={'fontsize': '12', 'fontweight' : '3'})
    plot_number = plot_number + 1
plt

There are a few things that are still wrong with this map... the max and min for each map has been manually set. Ideally, we would want to find the largest count of deaths for a given year+county combo and use that as the max. 

Also, we're using just the count of deaths instead of a rate which would be much more useful in determining changes over time.

In [ ]:
deaths_by_cnty = df[['INJURY_FIPS','STATEID','death_year']].groupby(['INJURY_FIPS','death_year']).count().reset_index().sort_values('STATEID', ascending=False)
deaths_by_cnty['INJURY_FIPS'] = deaths_by_cnty['INJURY_FIPS'].astype(int).astype(str)
deaths_by_cnty.rename(columns={'STATEID': 'death_count'}, inplace=True)

# add county pop data
cnty_pop = pd.read_csv('data/mn_cnty_pop_estimates.csv')
cnty_pop['cnty_fips'] = cnty_pop['GEO_ID'].apply(lambda x: x[-5:])

# create a new dataframe
cnty_death_pop_merge = deaths_by_cnty.merge(cnty_pop, how='left',
                                            left_on='INJURY_FIPS',
                                            right_on='cnty_fips')

# calculate death rate column per 1,000 residents
cnty_death_pop_merge['death_rate'] = (cnty_death_pop_merge['death_count'] / cnty_death_pop_merge['total_pop'])*1000

# replace those na values with 0s
cnty_death_pop_merge.fillna(value={'death_rate': 0}, inplace=True)

by_yearcnty = pd.pivot_table(cnty_death_pop_merge, values='death_rate', index='INJURY_FIPS',
                            columns=['death_year'], aggfunc='sum').reset_index()

yr_geodeaths = mn_cnty.merge(by_yearcnty, left_on='GEOID', right_on='INJURY_FIPS', how='left')

yr_geodeaths = yr_geodeaths.fillna({2005:0,2006:0,2007:0,2008:0,2009:0,2010:0,
                                   2011:0,2012:0,2013:0,2014:0,2015:0,
                                   2016:0,2017:0})

In [ ]:
yr_geodeaths.sort_values(2017, ascending=False)

In [ ]:
plt.figure(figsize=(20,15), facecolor='white')

plot_number = 1
for year in year_list:
    year = year
    # Inside of an image that's a 15x13 grid, put this
    # graph in the in the plot_number slot.
    ax = plt.subplot(4, 4, plot_number)
    vmin = 0#yr_geodeaths[year].min
    vmax = 0.3#yr_geodeaths[year].max
    
    yr_geodeaths.plot(column=year, cmap='Greens', linewidth=0.8, vmin=vmin, 
                      vmax=vmax, edgecolor='0.8',ax=ax, legend=True)
    
    ax.axis('off')
    
    ax.set_title(year, fontdict={'fontsize': '12', 'fontweight' : '3'})
    plot_number = plot_number + 1
plt